# Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [99]:
TRAINING_DATA_FILE='SpamData/02_Training/train-data-bidon.txt'
TESTING_DATA_FILE='SpamData/03_Testing/test-data-bidon.txt'

TOKEN_SPAM_PROB_FILE='SpamData/03_Testing/prob-spam-bidon.txt'
TOKEN_HAM_PROB_FILE='SpamData/03_Testing/prob-ham-bidon.txt'
TOKEN_ALL_PROB_FILE='SpamData/03_Testing/prob-all-tokens-bidon.txt'

TEST_FEATURE_MATRIX='SpamData/03_Testing/test-features-bidon.txt'
TEST_TARGET_FILE='SpamData/03_Testing/test-target-bidon.txt'

VOCAB_SIZE=2500

# Read and load from .txt to np arrays

In [3]:
sparse_train_data=np.loadtxt(TRAINING_DATA_FILE,delimiter=' ',dtype=int)
sparse_test_data=np.loadtxt(TESTING_DATA_FILE,delimiter=' ',dtype=int)

In [8]:
sparse_train_data.shape

(265591, 4)

In [9]:
print('Nr of emails in training file', np.unique(sparse_train_data[:,0]).size)

Nr of emails in training file 4022


In [10]:
print('Nr of emails in testing file', np.unique(sparse_test_data[:,0]).size)

Nr of emails in testing file 1717


In [11]:
sparse_test_data

array([[   8,    0,    1,    2],
       [   8,    1,    1,    5],
       [   8,    2,    1,    1],
       ...,
       [5793, 1326,    0,    1],
       [5793, 2059,    0,    1],
       [5793, 2356,    0,    1]])

In [12]:
sparse_train_data

array([[   0,    0,    1,    2],
       [   0,    2,    1,    1],
       [   0,   16,    1,    2],
       ...,
       [5795, 1200,    0,    1],
       [5795, 1857,    0,    1],
       [5795, 2081,    0,    1]])

### How to create empty df

In [13]:
column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [14]:
len(column_names)

2502

In [16]:
index_names=np.unique(sparse_train_data[:,0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [20]:
full_train_data=pd.DataFrame(index=index_names,columns=column_names)
full_train_data.fillna(value=0,inplace=True)

In [22]:
full_train_data.head()

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create full matrix from sparse matrix

In [27]:
def make_full_matrix(sparse_matrix,nr_words,doc_idx=0,word_idx=1,cat_idx=2,freq_idx=3):
    """
    Form full matrix from sparse matrix.Returns a df.
    
    Keyword args:
    sparse_matrix--np array
    nr_words--size of the vocab. Total nr of tokens
    doc_idx-- pos of doc id in sparse matrix.Default 1st col
    word_idx--pos of word id in sparse matrix.Default 2nd col
    cat_idx--pos of label in sparse matrix.Default 3rd col
    freq_idx--pos of occurences in sparse matrix.Default 4th col
    """
    column_names=['DOC_ID']+['CATEGORY']+list(range(0,VOCAB_SIZE))
    doc_id_names=np.unique(sparse_matrix[:,0])
    full_matrix=pd.DataFrame(index=doc_id_names,columns=column_names)
    full_matrix.fillna(value=0,inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr=sparse_matrix[i][doc_idx]
        word_id=sparse_matrix[i][word_idx]
        label=sparse_matrix[i][cat_idx]
        occurence=sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr,'DOC_ID']=doc_nr
        full_matrix.at[doc_nr,'CATEGORY']=label
        full_matrix.at[doc_nr,word_id]=occurence
    
    full_matrix.set_index('DOC_ID',inplace=True)
    return full_matrix
    

In [28]:
%%time
full_train_data=make_full_matrix(sparse_train_data,VOCAB_SIZE)

CPU times: user 9.81 s, sys: 283 ms, total: 10.1 s
Wall time: 9.52 s


In [30]:
full_train_data.tail()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
5789,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,3,3,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5795,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model
## Calculating the prob of spam

In [33]:
full_train_data.CATEGORY.size

4022

In [34]:
full_train_data.CATEGORY.sum()

1257

In [35]:
prob_spam=full_train_data.CATEGORY.sum()/full_train_data.CATEGORY.shape[0]

In [36]:
prob_spam

0.3125310790651417

## Total nr of words/tokens

In [37]:
full_train_features=full_train_data.loc[:,full_train_data.columns!='CATEGORY']
full_train_features.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,2,5,2,0,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


In [38]:
email_lengths=full_train_features.sum(axis=1)
email_lengths.shape

(4022,)

In [39]:
email_lengths[:5]

DOC_ID
0     73
1     71
2     51
3     35
4    198
dtype: int64

In [40]:
total_wc=email_lengths.sum()
total_wc

444968

## Nr  of token in spam and ham emails

In [50]:
spam_lengths_ar=full_train_data.loc[full_train_data.CATEGORY==1,full_train_data.columns!='CATEGORY']
spam_lengths_ar.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,2,5,2,0,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0


In [51]:
spam_lengths_ar.shape

(1257, 2500)

In [52]:
spam_lengths=spam_lengths_ar.sum(axis=1 )

In [88]:
spam_lens=spam_lengths.sum()
spam_lens

189899

In [55]:
ham_lengths=email_lengths[full_train_data.CATEGORY==0]
ham_lengths.shape

(2765,)

In [56]:
email_lengths.shape[0]-spam_lengths_ar.shape[0]-ham_lengths.shape[0]

0

In [57]:
non_spam_wc=ham_lengths.sum()
non_spam_wc

255069

In [63]:
print('Avg nr of words in spam emails: {:.0f}'.format(spam_lens/spam_lengths_ar.shape[0]))
print('Avg nr of words in ham emails: {:.0f}'.format(non_spam_wc/ham_lengths.shape[0]))

Avg nr of words in spam emails: 151
Avg nr of words in ham emails: 92


## Summing the tokens occuring in spam

In [68]:
full_train_features[full_train_data.CATEGORY==1]

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,3,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,2,5,2,0,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1885,1,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1887,0,2,3,1,0,3,2,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1889,2,2,1,0,1,1,2,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [69]:
train_spam_tokens=full_train_features[full_train_data.CATEGORY==1]

In [72]:
train_spam_tokens.shape

(1257, 2500)

In [76]:
summed_spam_tokens=train_spam_tokens.sum() +1

In [77]:
summed_spam_tokens.shape

(2500,)

In [78]:
summed_spam_tokens.tail()

2495    17
2496    22
2497     8
2498    13
2499     4
dtype: int64

In [79]:
train_ham_tokens=full_train_features[full_train_data.CATEGORY==0]
train_ham_tokens.shape

(2765, 2500)

In [83]:
summed_ham_tokens=train_ham_tokens.sum()+1

In [85]:
summed_ham_tokens.tail()

2495    17
2496    12
2497    25
2498    11
2499    16
dtype: int64

In [87]:
train_ham_tokens[2499].sum()

15

## P(Token | Spam) - Probability that a token occurs given the email is spam

In [89]:
prob_token_spam=summed_spam_tokens / (spam_lens+VOCAB_SIZE)
prob_token_spam.head()

0    0.009252
1    0.005062
2    0.007245
3    0.011455
4    0.006736
dtype: float64

In [90]:
prob_token_spam.sum()

1.0

## P(Token | Ham) - Probability that a token occurs given the email is ham

In [91]:
prob_token_ham=summed_ham_tokens / (non_spam_wc+VOCAB_SIZE)
prob_token_ham.head()

0    0.019727
1    0.009768
2    0.007850
3    0.003525
4    0.006464
dtype: float64

In [92]:
prob_token_ham.sum()

1.0

## P(Token)- Prob that Token Occurs

In [93]:
prob_tokens_all=full_train_features.sum()/total_wc
prob_tokens_all.head()

0    0.015415
1    0.007839
2    0.007672
3    0.006989
4    0.006650
dtype: float64

In [94]:
prob_tokens_all.sum()

1.0

# SAve the trained Model

In [98]:
np.savetxt(TOKEN_SPAM_PROB_FILE,prob_token_spam)
np.savetxt(TOKEN_HAM_PROB_FILE,prob_token_ham)
np.savetxt(TOKEN_ALL_PROB_FILE,prob_tokens_all)

# Prepare test data

In [100]:
%%time
full_test_data=make_full_matrix(sparse_test_data,VOCAB_SIZE)

CPU times: user 4.33 s, sys: 92.7 ms, total: 4.42 s
Wall time: 3.79 s


In [101]:
full_test_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,2,5,1,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,1,2,0,2,1,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,1,0,0,0,0,0,0,0,2,0,...,0,0,0,0,0,0,0,0,0,0
17,1,0,0,3,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [103]:
X_test=full_test_data.loc[:,full_test_data.columns!='CATEGORY']
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,2,5,1,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12,2,0,2,1,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15,0,0,0,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
17,0,0,3,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
y_test=full_test_data.CATEGORY

In [105]:
np.savetxt(TEST_TARGET_FILE,y_test)
np.savetxt(TEST_FEATURE_MATRIX,X_test)